# Quranic Corpus Web Scraper 

This notebook scrapes all occurrences of "Allah" from the Quranic Arabic Corpus website.

**Data source:** https://corpus.quran.com/search.jsp?q=lem%3A%7Bll%7Eah+pos%3Apn

**Total results:** Exactly 2,699 UNIQUE occurrences (no duplicates)

## Error handling:
**DUPLICATES ARE REJECTED, NOT CLEANED**
- During scraping: If a location is seen twice, the second one is REJECTED immediately
- No duplicate ever enters the dataset
- Validation confirms zero duplicates exist
- Report shows how many duplicates were rejected

## 1. Install Required Libraries

In [1]:
!pip install requests beautifulsoup4 pandas matplotlib

## 2. Import Libraries

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import os
from IPython.display import display, HTML

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


## 3. Define Scraping Function

In [3]:
def scrape_page(page_num, verbose=True):
    """
    Scrape a single page of results from the Quranic corpus.
    
    Returns all results found on the page (duplicates handled by caller).
    """
    url = f"https://corpus.quran.com/search.jsp?q=lem%3A%7Bll%7Eah+pos%3Apn&s=1&page={page_num}"
    
    if verbose:
        print(f"Scraping page {page_num}...", end=" ")
    
    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        if verbose:
            print("✓")
    except requests.RequestException as e:
        if verbose:
            print(f"✗ Error: {e}")
        return []
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    results = []
    rows = soup.find_all('tr')
    
    for row in rows:
        c1 = row.find('td', class_='c1')
        c2 = row.find('td', class_='c2')
        c3 = row.find('td', class_='c3')
        
        if c1 and c2 and c3:
            location_span = c1.find('span', class_='l')
            if not location_span:
                continue
                
            location_text = location_span.get_text(strip=True)
            
            if not (location_text.startswith('(') and location_text.endswith(')')):
                continue
                
            location = location_text.strip('()')
            
            # Validate location format
            match = re.match(r'^(\d+):(\d+):(\d+)$', location)
            if not match:
                continue
            
            chapter = match.group(1)
            verse = match.group(2)
            word_position = match.group(3)
            
            transliteration_tag = c1.find('i', class_='ab')
            transliteration = transliteration_tag.get_text(strip=True) if transliteration_tag else ""
            
            if not transliteration:
                continue
            
            translation_link = c2.find('a')
            translation = translation_link.get_text(strip=True) if translation_link else c2.get_text(strip=True)
            
            arabic_text = c3.get_text(strip=True)
            
            if not arabic_text:
                continue
            
            results.append({
                'location': location,
                'chapter': chapter,
                'verse': verse,
                'word_position': word_position,
                'transliteration': transliteration,
                'translation': translation,
                'arabic_verse': arabic_text
            })
    
    return results

## 4. Main Scraping Function - REJECTS DUPLICATES

In [4]:
def scrape_all_pages(total_pages=54):
    """
    Scrape all pages and REJECT duplicates immediately.
    
    CRITICAL: Duplicates are NEVER added to the results list.
    They are counted and reported, but REJECTED.
    """
    unique_results = []  # Only unique entries go here
    seen_locations = set()  # Track what we've already added
    
    # Track duplicate encounters for reporting
    duplicate_locations = []
    
    print(f"Starting scrape of {total_pages} pages...")
    print("=" * 60)
    print("DUPLICATE POLICY: Reject on sight - never add to dataset")
    print("=" * 60)
    
    for page in range(1, total_pages + 1):
        page_results = scrape_page(page)
        
        # Process each result
        for result in page_results:
            loc = result['location']
            
            if loc not in seen_locations:
                # First time seeing this - ADD IT
                seen_locations.add(loc)
                unique_results.append(result)
            else:
                # Already seen - REJECT IT
                duplicate_locations.append({
                    'location': loc,
                    'page': page,
                    'rejected': True
                })
        
        if page % 10 == 0:
            print(f"Progress: {page}/{total_pages} pages | Unique: {len(unique_results)} | Rejected: {len(duplicate_locations)}")
        
        time.sleep(0.5)
    
    print("=" * 60)
    print(f"\n✓ SCRAPING COMPLETE")
    print("=" * 60)
    print(f"Unique entries collected: {len(unique_results)}")
    print(f"Duplicate entries REJECTED: {len(duplicate_locations)}")
    print(f"Expected unique entries: 2699")
    print(f"Difference: {len(unique_results) - 2699:+d}")
    
    # Report duplicates if found
    if duplicate_locations:
        print(f"\n⚠️  {len(duplicate_locations)} DUPLICATES WERE REJECTED:")
        print("-" * 60)
        for i, dup in enumerate(duplicate_locations[:20], 1):  # Show first 20
            print(f"  {i}. Location {dup['location']} (rejected on page {dup['page']})")
        if len(duplicate_locations) > 20:
            print(f"  ... and {len(duplicate_locations) - 20} more")
        print("-" * 60)
        print("These duplicates were NOT added to the dataset.")
    else:
        print("\n✓ NO DUPLICATES ENCOUNTERED")
    
    print("=" * 60)
    
    return unique_results, duplicate_locations

## 5. Validation Function - Verify Zero Duplicates

In [5]:
def validate_no_duplicates(df):
    """
    Validate that the DataFrame has ZERO duplicates.
    This should always pass if scraping worked correctly.
    """
    print("\n" + "=" * 60)
    print("VALIDATION: Checking for duplicates in final dataset")
    print("=" * 60)
    
    # Check for duplicate locations
    location_counts = df['location'].value_counts()
    duplicates = location_counts[location_counts > 1]
    
    if len(duplicates) == 0:
        print("✓ VALIDATION PASSED: Zero duplicates found")
        print(f"  All {len(df)} entries have unique locations")
        print("  Dataset is clean and ready for analysis")
        return True
    else:
        print("✗ VALIDATION FAILED: Duplicates found in dataset!")
        print(f"  {len(duplicates)} locations appear multiple times")
        print("\nThis should not happen. Duplicate locations:")
        for loc, count in duplicates.items():
            print(f"  {loc}: {count} times")
        return False

## 6. Test with First 3 Pages

In [6]:
print("Testing with first 3 pages...\n")
test_results, test_duplicates = scrape_all_pages(total_pages=3)

expected_per_page = 2699 / 54
expected_for_3 = expected_per_page * 3

print(f"\nExpected approximately: {expected_for_3:.0f} results")
print(f"Actual unique results: {len(test_results)}")

if len(test_results) > 0:
    df_test = pd.DataFrame(test_results)
    validate_no_duplicates(df_test)
    
    print("\nSample data:")
    display(df_test.head(10))

Testing with first 3 pages...

Starting scrape of 3 pages...
DUPLICATE POLICY: Reject on sight - never add to dataset
Scraping page 1... ✓
Scraping page 2... ✓
Scraping page 3... ✓

✓ SCRAPING COMPLETE
Unique entries collected: 150
Duplicate entries REJECTED: 3
Expected unique entries: 2699
Difference: -2549

⚠️  3 DUPLICATES WERE REJECTED:
------------------------------------------------------------
  1. Location 1:1:2 (rejected on page 1)
  2. Location 2:97:11 (rejected on page 2)
  3. Location 2:161:10 (rejected on page 3)
------------------------------------------------------------
These duplicates were NOT added to the dataset.

Expected approximately: 150 results
Actual unique results: 150

VALIDATION: Checking for duplicates in final dataset
✓ VALIDATION PASSED: Zero duplicates found
  All 150 entries have unique locations
  Dataset is clean and ready for analysis

Sample data:


,location,chapter,verse,word_position,transliteration,translation,arabic_verse
0,1:1:2,1,1,2,l-lahi,(of) Allah,بِسْمِاللَّهِالرَّحْمَٰنِ الرَّحِيمِ
1,1:2:2,1,2,2,lillahi,(be) to Allah,الْحَمْدُلِلَّهِرَبِّ الْعَالَمِينَ
2,2:7:2,2,7,2,l-lahu,Allah,خَتَمَاللَّهُعَلَىٰ قُلُوبِهِمْ وَعَلَىٰ سَمْع...
3,2:8:6,2,8,6,bil-lahi,in Allah,وَمِنَ النَّاسِ مَنْ يَقُولُ آمَنَّابِاللَّهِو...
4,2:9:2,2,9,2,l-laha,Allah,يُخَادِعُونَاللَّهَوَالَّذِينَ آمَنُوا وَمَا ي...
5,2:10:5,2,10,5,l-lahu,Allah,فِي قُلُوبِهِمْ مَرَضٌ فَزَادَهُمُاللَّهُمَرَضًا
6,2:15:1,2,15,1,al-lahu,Allah,اللَّهُيَسْتَهْزِئُ بِهِمْ وَيَمُدُّهُمْ فِي ط...
7,2:17:11,2,17,11,l-lahu,Allah,فَلَمَّا أَضَاءَتْ مَا حَوْلَهُ ذَهَبَاللَّهُب...
8,2:19:17,2,19,17,wal-lahu,And Allah,وَاللَّهُمُحِيطٌ بِالْكَافِرِينَ
9,2:20:16,2,20,16,l-lahu,Allah,وَلَوْ شَاءَاللَّهُلَذَهَبَ بِسَمْعِهِمْ وَأَب...


## 7. Scrape All 54 Pages (Full Dataset)

**Warning:** This will take approximately 30-45 seconds.

In [7]:
# Scrape all pages
all_results, rejected_duplicates = scrape_all_pages(total_pages=54)

# Convert to DataFrame
df = pd.DataFrame(all_results)

print(f"\nFinal DataFrame shape: {df.shape}")
print(f"Columns: {list(df.columns)}")

Starting scrape of 54 pages...
DUPLICATE POLICY: Reject on sight - never add to dataset
Scraping page 1... ✓
Scraping page 2... ✓
Scraping page 3... ✓
Scraping page 4... ✓
Scraping page 5... ✓
Scraping page 6... ✓
Scraping page 7... ✓
Scraping page 8... ✓
Scraping page 9... ✓
Scraping page 10... ✓
Progress: 10/54 pages | Unique: 500 | Rejected: 10
Scraping page 11... ✓
Scraping page 12... ✓
Scraping page 13... ✓
Scraping page 14... ✓
Scraping page 15... ✓
Scraping page 16... ✓
Scraping page 17... ✓
Scraping page 18... ✓
Scraping page 19... ✓
Scraping page 20... ✓
Progress: 20/54 pages | Unique: 1000 | Rejected: 20
Scraping page 21... ✓
Scraping page 22... ✓
Scraping page 23... ✓
Scraping page 24... ✓
Scraping page 25... ✓
Scraping page 26... ✓
Scraping page 27... ✓
Scraping page 28... ✓
Scraping page 29... ✓
Scraping page 30... ✓
Progress: 30/54 pages | Unique: 1500 | Rejected: 30
Scraping page 31... ✓
Scraping page 32... ✓
Scraping page 33... ✓
Scraping page 34... ✓
Scraping page 35..

## 8. Final Validation

In [8]:
# Validate the final dataset
validation_passed = validate_no_duplicates(df)

if validation_passed:
    print("\n✓ Dataset ready for export")
else:
    print("\n⚠️  Dataset has issues - review above")


VALIDATION: Checking for duplicates in final dataset
✓ VALIDATION PASSED: Zero duplicates found
  All 2699 entries have unique locations
  Dataset is clean and ready for analysis

✓ Dataset ready for export


## 9. Save to CSV

In [9]:
if len(df) > 0:
    csv_filename = 'allah_occurrences_NO_DUPLICATES.csv'
    df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
    
    print("=" * 60)
    print("FILE SAVED")
    print("=" * 60)
    print(f"Filename: {csv_filename}")
    print(f"Total rows: {len(df)}")
    print(f"File size: {os.path.getsize(csv_filename) / 1024:.1f} KB")
    print(f"Duplicates in file: 0 (guaranteed)")
    print("=" * 60)
else:
    print("⚠️  No data to save")

FILE SAVED
Filename: allah_occurrences_NO_DUPLICATES.csv
Total rows: 2699
File size: 420.2 KB
Duplicates in file: 0 (guaranteed)


## 10. Export Rejected Duplicates Report (Optional)

In [10]:
if rejected_duplicates:
    df_rejected = pd.DataFrame(rejected_duplicates)
    rejected_filename = 'rejected_duplicates_report.csv'
    df_rejected.to_csv(rejected_filename, index=False)
    
    print(f"\n📋 Rejected duplicates report saved: {rejected_filename}")
    print(f"   Total rejected: {len(rejected_duplicates)}")
    print("\nFirst 10 rejected entries:")
    display(df_rejected.head(10))
else:
    print("\n✓ No duplicates were encountered, so no rejection report needed")


📋 Rejected duplicates report saved: rejected_duplicates_report.csv
   Total rejected: 54

First 10 rejected entries:


,location,page,rejected
0,1:1:2,1,True
1,2:97:11,2,True
2,2:161:10,3,True
3,2:209:10,4,True
4,2:235:15,5,True
5,2:264:16,6,True
6,3:19:22,7,True
7,3:73:11,8,True
8,3:120:18,9,True
9,3:163:4,10,True


## 11. Data Analysis

In [11]:
if len(df) > 0:
    print("=" * 60)
    print("DATASET STATISTICS")
    print("=" * 60)
    print(f"Total unique occurrences: {len(df)}")
    print(f"Unique chapters: {df['chapter'].nunique()}")
    print(f"Unique verses: {len(df.groupby(['chapter', 'verse']))}")
    print(f"Unique transliterations: {df['transliteration'].nunique()}")
    print(f"Duplicates in dataset: 0 (verified)")
    
    print("\n" + "=" * 60)
    print("TOP 10 TRANSLITERATIONS")
    print("=" * 60)
    display(df['transliteration'].value_counts().head(10))
    
    print("\n" + "=" * 60)
    print("TOP 10 CHAPTERS BY FREQUENCY")
    print("=" * 60)
    display(df['chapter'].value_counts().head(10))
else:
    print("⚠️  No data available")

DATASET STATISTICS
Total unique occurrences: 2699
Unique chapters: 85
Unique verses: 1821
Unique transliterations: 17
Duplicates in dataset: 0 (verified)

TOP 10 TRANSLITERATIONS


transliteration
l-lahi       827
l-lahu       701
l-laha       591
wal-lahu     239
bil-lahi     139
lillahi      116
al-lahu       32
walillahi     27
tal-lahi       8
fal-lahu       6
Name: count, dtype: int64


TOP 10 CHAPTERS BY FREQUENCY


chapter
2     282
4     229
3     209
9     169
5     147
33     90
8      88
6      87
16     84
24     80
Name: count, dtype: int64

## 12. Sample Data

In [12]:
if len(df) > 0:
    print("First 10 results:")
    display(df.head(10))
    
    print("\nLast 10 results:")
    display(df.tail(10))
else:
    print("⚠️  No data available")

First 10 results:


,location,chapter,verse,word_position,transliteration,translation,arabic_verse
0,1:1:2,1,1,2,l-lahi,(of) Allah,بِسْمِاللَّهِالرَّحْمَٰنِ الرَّحِيمِ
1,1:2:2,1,2,2,lillahi,(be) to Allah,الْحَمْدُلِلَّهِرَبِّ الْعَالَمِينَ
2,2:7:2,2,7,2,l-lahu,Allah,خَتَمَاللَّهُعَلَىٰ قُلُوبِهِمْ وَعَلَىٰ سَمْع...
3,2:8:6,2,8,6,bil-lahi,in Allah,وَمِنَ النَّاسِ مَنْ يَقُولُ آمَنَّابِاللَّهِو...
4,2:9:2,2,9,2,l-laha,Allah,يُخَادِعُونَاللَّهَوَالَّذِينَ آمَنُوا وَمَا ي...
5,2:10:5,2,10,5,l-lahu,Allah,فِي قُلُوبِهِمْ مَرَضٌ فَزَادَهُمُاللَّهُمَرَضًا
6,2:15:1,2,15,1,al-lahu,Allah,اللَّهُيَسْتَهْزِئُ بِهِمْ وَيَمُدُّهُمْ فِي ط...
7,2:17:11,2,17,11,l-lahu,Allah,فَلَمَّا أَضَاءَتْ مَا حَوْلَهُ ذَهَبَاللَّهُب...
8,2:19:17,2,19,17,wal-lahu,And Allah,وَاللَّهُمُحِيطٌ بِالْكَافِرِينَ
9,2:20:16,2,20,16,l-lahu,Allah,وَلَوْ شَاءَاللَّهُلَذَهَبَ بِسَمْعِهِمْ وَأَب...



Last 10 results:


,location,chapter,verse,word_position,transliteration,translation,arabic_verse
2689,95:8:2,95,8,2,l-lahu,Allah,أَلَيْسَاللَّهُبِأَحْكَمِ الْحَاكِمِينَ
2690,96:14:4,96,14,4,l-laha,Allah,أَلَمْ يَعْلَمْ بِأَنَّاللَّهَيَرَىٰ
2691,98:2:3,98,2,3,l-lahi,Allah,رَسُولٌ مِنَاللَّهِيَتْلُو صُحُفًا مُطَهَّرَةً
2692,98:5:5,98,5,5,l-laha,Allah,وَمَا أُمِرُوا إِلَّا لِيَعْبُدُوااللَّهَمُخْل...
2693,98:8:14,98,8,14,l-lahu,Allah,رَضِيَاللَّهُعَنْهُمْ وَرَضُوا عَنْهُ ذَٰلِكَ ...
2694,104:6:2,104,6,2,l-lahi,Allah,نَارُاللَّهِالْمُوقَدَةُ
2695,110:1:4,110,1,4,l-lahi,(of) Allah,إِذَا جَاءَ نَصْرُاللَّهِوَالْفَتْحُ
2696,110:2:6,110,2,6,l-lahi,(of) Allah,وَرَأَيْتَ النَّاسَ يَدْخُلُونَ فِي دِينِاللَّ...
2697,112:1:3,112,1,3,l-lahu,(is) Allah,قُلْ هُوَاللَّهُأَحَدٌ
2698,112:2:1,112,2,1,al-lahu,Allah,اللَّهُالصَّمَدُ


## Summary

### ✓ What This Notebook Does:
1. **Scrapes** all 54 pages of Allah occurrences
2. **Rejects** duplicate locations immediately (never adds them)
3. **Reports** how many duplicates were rejected
4. **Validates** the final dataset has zero duplicates
5. **Saves** a clean CSV with exactly 2,699 unique entries

### ✓ Guarantees:
- **Zero duplicates** in the final CSV file
- **Exactly 2,699** unique location entries (or very close)
- **No NameError** (os module imported)
- **Transparent reporting** of what was rejected and why

### ✓ Philosophy:
**Duplicates are REJECTED, not cleaned**
- If we see a location twice, the second one is thrown away immediately
- No duplicate ever makes it into `unique_results`
- The validation step should always show zero duplicates
- You get a clean dataset from the start, not a messy one that needs cleaning